# 📊 Module 12: Agent Evaluation & Observability

**AI Agent Architectures Workshop - Day 2**

This notebook covers:
- Measuring tool call accuracy
- Measuring goal accuracy
- Azure Monitor integration
- Application Insights for agent workflows
- Safe interaction design

**Prerequisites:** Run `00_setup.ipynb` first.

In [ ]:
!pip install openai python-dotenv opencensus-ext-azure --quiet

In [ ]:
import os
import json
import time
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Optional

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

## 1. Tool Call Accuracy Measurement

In [ ]:
@dataclass
class ToolCallEvaluation:
    """Evaluation result for a single tool call"""
    query: str
    expected_tool: str
    actual_tool: Optional[str]
    expected_params: Dict
    actual_params: Optional[Dict]
    tool_correct: bool
    params_correct: bool
    latency_ms: float

class ToolCallEvaluator:
    """Evaluate agent tool calling accuracy"""
    
    def __init__(self):
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_account_balance",
                    "description": "Get the current balance of a bank account",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "account_id": {"type": "string", "description": "The account ID"}
                        },
                        "required": ["account_id"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "transfer_funds",
                    "description": "Transfer money between accounts",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "from_account": {"type": "string"},
                            "to_account": {"type": "string"},
                            "amount": {"type": "number"}
                        },
                        "required": ["from_account", "to_account", "amount"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_transaction_history",
                    "description": "Get recent transactions for an account",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "account_id": {"type": "string"},
                            "days": {"type": "integer", "default": 30}
                        },
                        "required": ["account_id"]
                    }
                }
            }
        ]
        self.evaluations: List[ToolCallEvaluation] = []
    
    def evaluate(self, query: str, expected_tool: str, expected_params: Dict) -> ToolCallEvaluation:
        """Evaluate a single tool call"""
        start_time = time.time()
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": query}],
            tools=self.tools,
            tool_choice="auto"
        )
        
        latency_ms = (time.time() - start_time) * 1000
        
        # Extract actual tool call
        actual_tool = None
        actual_params = None
        
        if response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            actual_tool = tool_call.function.name
            actual_params = json.loads(tool_call.function.arguments)
        
        # Evaluate correctness
        tool_correct = actual_tool == expected_tool
        params_correct = self._compare_params(expected_params, actual_params)
        
        evaluation = ToolCallEvaluation(
            query=query,
            expected_tool=expected_tool,
            actual_tool=actual_tool,
            expected_params=expected_params,
            actual_params=actual_params,
            tool_correct=tool_correct,
            params_correct=params_correct,
            latency_ms=latency_ms
        )
        
        self.evaluations.append(evaluation)
        return evaluation
    
    def _compare_params(self, expected: Dict, actual: Optional[Dict]) -> bool:
        """Compare expected and actual parameters"""
        if actual is None:
            return False
        for key, value in expected.items():
            if key not in actual or actual[key] != value:
                return False
        return True
    
    def get_metrics(self) -> Dict:
        """Calculate evaluation metrics"""
        if not self.evaluations:
            return {}
        
        total = len(self.evaluations)
        tool_correct = sum(1 for e in self.evaluations if e.tool_correct)
        params_correct = sum(1 for e in self.evaluations if e.params_correct)
        avg_latency = sum(e.latency_ms for e in self.evaluations) / total
        
        return {
            "total_evaluations": total,
            "tool_accuracy": tool_correct / total,
            "param_accuracy": params_correct / total,
            "avg_latency_ms": avg_latency
        }

print("✅ Tool call evaluator defined")

In [ ]:
# Run tool call evaluation
evaluator = ToolCallEvaluator()

test_cases = [
    {
        "query": "What's the balance in account A123?",
        "expected_tool": "get_account_balance",
        "expected_params": {"account_id": "A123"}
    },
    {
        "query": "Transfer $500 from account A123 to account B456",
        "expected_tool": "transfer_funds",
        "expected_params": {"from_account": "A123", "to_account": "B456", "amount": 500}
    },
    {
        "query": "Show me the last 7 days of transactions for account A123",
        "expected_tool": "get_transaction_history",
        "expected_params": {"account_id": "A123", "days": 7}
    }
]

print("=== Tool Call Evaluation ===")
for tc in test_cases:
    result = evaluator.evaluate(tc["query"], tc["expected_tool"], tc["expected_params"])
    status = "✅" if result.tool_correct and result.params_correct else "❌"
    print(f"{status} Query: {tc['query'][:40]}...")
    print(f"   Expected: {tc['expected_tool']} | Actual: {result.actual_tool}")
    print(f"   Tool correct: {result.tool_correct} | Params correct: {result.params_correct}")

metrics = evaluator.get_metrics()
print(f"\n📊 Metrics:")
print(f"   Tool Accuracy: {metrics['tool_accuracy']:.1%}")
print(f"   Param Accuracy: {metrics['param_accuracy']:.1%}")
print(f"   Avg Latency: {metrics['avg_latency_ms']:.0f}ms")

## 2. Goal Accuracy Measurement

In [ ]:
@dataclass
class GoalEvaluation:
    """Evaluation of whether agent achieved the goal"""
    goal: str
    agent_response: str
    goal_achieved: bool
    reasoning: str
    score: float  # 0-1

class GoalEvaluator:
    """Evaluate whether agent responses achieve stated goals"""
    
    def evaluate(self, goal: str, agent_response: str) -> GoalEvaluation:
        """Use LLM-as-judge to evaluate goal achievement"""
        
        eval_prompt = f"""Evaluate whether the agent's response achieves the stated goal.

GOAL: {goal}

AGENT RESPONSE: {agent_response}

Evaluate on these criteria:
1. Did the agent address the goal directly?
2. Was the response accurate and helpful?
3. Were there any errors or hallucinations?

Respond in JSON format:
{{
    "goal_achieved": true/false,
    "score": 0.0-1.0,
    "reasoning": "explanation"
}}"""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": eval_prompt}],
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        
        return GoalEvaluation(
            goal=goal,
            agent_response=agent_response,
            goal_achieved=result["goal_achieved"],
            reasoning=result["reasoning"],
            score=result["score"]
        )

# Test goal evaluation
goal_evaluator = GoalEvaluator()

test_goals = [
    {
        "goal": "Explain how to dispute a transaction",
        "response": "To dispute a transaction, log into your online banking, go to the transaction, and click 'Dispute'. You have 60 days from the statement date to file a dispute."
    },
    {
        "goal": "Provide the current mortgage rate",
        "response": "I'm sorry, I don't have access to current rates. Please visit our website or call customer service."
    }
]

print("=== Goal Achievement Evaluation ===")
for test in test_goals:
    result = goal_evaluator.evaluate(test["goal"], test["response"])
    status = "✅" if result.goal_achieved else "❌"
    print(f"\n{status} Goal: {test['goal']}")
    print(f"   Score: {result.score:.1%}")
    print(f"   Reasoning: {result.reasoning[:100]}...")

## 3. Observability with Azure Monitor

In [ ]:
class AgentTelemetry:
    """Collect and report agent telemetry (simulated Azure Monitor)"""
    
    def __init__(self):
        self.metrics = []
        self.traces = []
        self.events = []
    
    def track_metric(self, name: str, value: float, properties: Dict = None):
        """Track a custom metric"""
        self.metrics.append({
            "timestamp": datetime.now().isoformat(),
            "name": name,
            "value": value,
            "properties": properties or {}
        })
    
    def track_trace(self, message: str, severity: str = "INFO", properties: Dict = None):
        """Track a trace message"""
        self.traces.append({
            "timestamp": datetime.now().isoformat(),
            "message": message,
            "severity": severity,
            "properties": properties or {}
        })
    
    def track_event(self, name: str, properties: Dict = None):
        """Track a custom event"""
        self.events.append({
            "timestamp": datetime.now().isoformat(),
            "name": name,
            "properties": properties or {}
        })
    
    def get_summary(self) -> Dict:
        """Get telemetry summary"""
        return {
            "total_metrics": len(self.metrics),
            "total_traces": len(self.traces),
            "total_events": len(self.events),
            "recent_metrics": self.metrics[-5:] if self.metrics else []
        }

# Instrumented agent
class InstrumentedAgent:
    """Agent with full observability"""
    
    def __init__(self):
        self.telemetry = AgentTelemetry()
    
    def invoke(self, query: str) -> str:
        """Invoke agent with telemetry"""
        start_time = time.time()
        
        # Track request event
        self.telemetry.track_event("agent_request", {"query_length": len(query)})
        self.telemetry.track_trace(f"Processing query: {query[:50]}...", "INFO")
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": query}]
            )
            
            result = response.choices[0].message.content
            
            # Track success metrics
            latency = (time.time() - start_time) * 1000
            tokens = response.usage.total_tokens
            
            self.telemetry.track_metric("latency_ms", latency)
            self.telemetry.track_metric("tokens_used", tokens)
            self.telemetry.track_event("agent_success", {
                "latency_ms": latency,
                "tokens": tokens
            })
            
            return result
            
        except Exception as e:
            # Track error
            self.telemetry.track_trace(f"Error: {str(e)}", "ERROR")
            self.telemetry.track_event("agent_error", {"error": str(e)})
            raise

# Test instrumented agent
agent = InstrumentedAgent()

queries = [
    "What is the interest rate on savings accounts?",
    "How do I open a new checking account?",
    "What are the requirements for a business loan?"
]

print("=== Instrumented Agent Execution ===")
for query in queries:
    response = agent.invoke(query)
    print(f"✅ Processed: {query[:40]}...")

summary = agent.telemetry.get_summary()
print(f"\n📊 Telemetry Summary:")
print(f"   Metrics: {summary['total_metrics']}")
print(f"   Traces: {summary['total_traces']}")
print(f"   Events: {summary['total_events']}")

## 4. Safe Interaction Design

In [ ]:
class SafetyGuard:
    """Safety checks for agent interactions"""
    
    def __init__(self):
        self.blocked_patterns = [
            "password", "ssn", "social security",
            "credit card number", "pin"
        ]
        self.high_risk_actions = [
            "transfer", "wire", "close account", "change password"
        ]
    
    def check_input(self, user_input: str) -> Dict:
        """Check user input for safety issues"""
        input_lower = user_input.lower()
        
        # Check for PII requests
        pii_detected = any(p in input_lower for p in self.blocked_patterns)
        
        # Check for high-risk actions
        high_risk = any(a in input_lower for a in self.high_risk_actions)
        
        return {
            "safe": not pii_detected,
            "requires_confirmation": high_risk,
            "pii_detected": pii_detected,
            "high_risk_action": high_risk
        }
    
    def check_output(self, agent_output: str) -> Dict:
        """Check agent output for safety issues"""
        output_lower = agent_output.lower()
        
        # Check for potential PII leakage
        pii_patterns = [
            r'\d{3}-\d{2}-\d{4}',  # SSN
            r'\d{16}',  # Credit card
            r'\d{4}-\d{4}-\d{4}-\d{4}'  # Credit card with dashes
        ]
        
        import re
        pii_found = any(re.search(p, agent_output) for p in pii_patterns)
        
        return {
            "safe": not pii_found,
            "pii_leaked": pii_found
        }

# Test safety guard
guard = SafetyGuard()

test_inputs = [
    "What is my account balance?",
    "What is my password?",
    "Transfer $5000 to account 12345",
    "Tell me my social security number"
]

print("=== Safety Guard Checks ===")
for inp in test_inputs:
    result = guard.check_input(inp)
    status = "✅" if result["safe"] else "🚫"
    confirm = "⚠️ Needs confirmation" if result["requires_confirmation"] else ""
    print(f"{status} {inp[:40]}... {confirm}")

## Summary

**Agent Evaluation Metrics:**

| Metric | Description | Target |
|--------|-------------|--------|
| Tool Call Accuracy | Correct tool selected | >95% |
| Parameter Accuracy | Correct parameters extracted | >90% |
| Goal Achievement | Task completed successfully | >85% |
| Latency P95 | 95th percentile response time | <3s |

**Azure Monitor Integration:**
- Application Insights for traces and metrics
- Custom events for agent actions
- Alerts for error rates and latency

**Safety Checklist:**
- Input validation for PII requests
- Output scanning for PII leakage
- Confirmation for high-risk actions
- Audit logging for compliance